In [1]:
from ultralytics import SAM
from PIL import Image
from package import pick_wall_point, distance_estimator

/Users/raimi/Playground/wall-segmentation/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IMAGE_PATH = "./assets/PHOTO-2025-04-27-17-10-54.jpg"

In [3]:
img_pil = Image.open(IMAGE_PATH).convert("RGB")
pt = pick_wall_point(img_pil)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [4]:
sam = SAM("sam2_b.pt")
results = sam.predict(source=IMAGE_PATH, points=[pt], save=False)


image 1/1 /Users/raimi/Playground/wall-segmentation/assets/PHOTO-2025-04-27-17-10-54.jpg: 1024x1024 1 0, 1313.0ms
Speed: 6.3ms preprocess, 1313.0ms inference, 1.6ms postprocess per image at shape (1, 3, 1024, 1024)


In [5]:
from depth_anything_v2.dpt import DepthAnythingV2
import torch

# 1) Load your model once, as before
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
model_configs = {
    'vits': {'encoder':'vits','features':64, 'out_channels':[48,96,192,384]},
    # ... your other configs ...
}
depth_anything = DepthAnythingV2(**model_configs[args.encoder])
depth_anything.load_state_dict(torch.load(
    f'checkpoints/depth_anything_v2_{args.encoder}.pth', map_location='cpu'))
depth_anything = depth_anything.to(DEVICE).eval()

# … later, inside your loop after you get `results` from SAM …
farthest, closest = distance_estimator(
    image_path=IMAGE_PATH,
    sam_results=results,
    depth_anything_model=depth_anything,
    input_size=args.input_size
)
print(f"Closest wall distance: {closest:.2f} m")
print(f"Farthest wall distance: {farthest:.2f} m")


Device set to use cpu


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed